In [58]:
# Library Import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import seaborn as sns
import os

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Imputer

# import the train test split
from sklearn.model_selection import train_test_split

In [59]:
# Get current working directory
cwd = os.getcwd()
print(cwd)

/Users/siddharthbhaduri/Desktop/Work/Fall-2017/Google-Project/Week7


In [60]:
price_file = 'IL_Corn_Price_1986_2016.csv'
price_df = pd.read_csv(price_file)


input_data = pd.read_csv('input_model_weighted_average.csv')
input_data.columns = map(str.upper, input_data.columns)

# drop UNNAMED:0.1
input_data = input_data.drop('UNNAMED: 0', axis=1)

input_data.head()

,YEAR,MONTH,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,HUM_HIGH,HUM_AVG,...,VIS_HIGH,VIS_AVG,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM
0,1986,JAN,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,75.092035,...,18.199743,9.979591,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000
1,1986,FEB,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,99.510367,82.286915,...,16.507919,6.647287,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457
2,1986,MAR,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,99.138166,65.295138,...,18.111402,10.209948,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313
3,1986,ARP,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,99.558871,58.069374,...,19.391184,11.580387,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521
4,1986,MAY,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,95.246992,62.031159,...,19.073952,9.481744,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441


In [61]:
grouped_data = input_data.groupby(["YEAR"])
year_names = input_data["YEAR"].drop_duplicates().reset_index(drop=True)
year_names = year_names[year_names != 2000] # Our database has incomplete data for 2000

month_names = grouped_data.get_group(1986)["MONTH"]
month_names
weather_factors = input_data.columns[2:]
colnames = []

for mon_name in month_names:
    for factors in weather_factors:
        colnames.append(mon_name + "_" + factors)

annual_weather_input = pd.DataFrame(0, index = year_names, columns = colnames)

In [62]:
for year in year_names:
    temp_df = grouped_data.get_group(year)
    temp_list = []
    for mon_name in month_names:
        temp_list.extend(temp_df.loc[temp_df["MONTH"] == mon_name].iloc[0,2:])
    annual_weather_input.loc[year] = temp_list

In [ ]:
temp_list

[56.208956699263105,
 28.210966908155012,
 1.8269073488677965,
 47.001875040320655,
 20.913060655064346,
 -8.8311724933041713,
 99.346199869725226,
 74.066579136591159,
 38.988302328920774,
 30.494711761976014,
 29.994242994184745,
 29.350997246749891,
 10.15259415262121,
 8.8438953578334019,
 0.059393619035925452,
 32.207612330731308,
 9.7781991637796661,
 1.2329440422206637,
 4.7998191553786205,
 8.511348052018322,
 2.4592319202202604,
 0.015259415262120647,
 66.900172312251428,
 33.807152508476591,
 6.3317052461588714,
 50.693124990895349,
 23.887684661398353,
 -6.1098053579121077,
 99.734310324375727,
 69.674338420202105,
 26.245278581878495,
 30.588809380612375,
 29.982752114132513,
 29.383167070046593,
 10.15259415262121,
 8.6546144728651146,
 0.061922114029921566,
 36.206539541291463,
 9.8244657204320127,
 0.83632979066558744,
 4.2247894481637722,
 6.8145302097922453,
 3.2992999294518683,
 0.56580018022030165,
 71.53824582798326,
 44.964953542219533,
 14.037543468262706,
 57.680

In [ ]:
annual_price_input = pd.DataFrame(0, index = year_names, columns = month_names)
annual_price_discounted_input = pd.DataFrame(0, index = year_names, columns = month_names)

grouped_price = price_df.groupby(["Year"])
month_names[3] = "APR"
for year in year_names:
    temp_df = grouped_price.get_group(year)
    temp_list_1 = []
    temp_list_2 = []
    for mon_name in month_names:
        temp_list_1.append(temp_df[temp_df["Period"] == mon_name]["Value"].iloc[0])
        temp_list_2.append(temp_df[temp_df["Period"] == mon_name]["Discounted_Value"].iloc[0])
    
    annual_price_input.loc[year] = temp_list_1
    annual_price_discounted_input.loc[year] = temp_list_2

In [ ]:
annual_price_input.to_csv("annual_price_input.csv")
annual_price_discounted_input.to_csv("annual_price_discounted_input.csv")
annual_weather_input.to_csv("annual_weather_input.csv")

In [ ]:
annual_weather_input

In [ ]:
annual_price_input

In [ ]:
annual_price_discounted_input

In [ ]:
df_oct_updated = pd.read_csv('annual_weather_input_nov.csv')
df_oct_updated.head()

In [ ]:
price_updated = pd.read_csv('annual_price_input_Sid.csv')
price_updated.head()

In [ ]:
df_oct_updated['NOV_PRICE'] = price_updated['NOV']
df_oct_updated

In [ ]:
# List of column names

#List to store results
# Stores the performance on test set.
model_results = []
#  Stores the name of model
model_name = []

# Making all column names UPPER CASE --> inline with the weather data file
df_oct_updated.columns = map(str.upper, df_oct_updated.columns)

# rename UNNAMED:0 to INDEX
#input_data.rename(columns={'UNNAMED: 0': 'INDEX'}, inplace=True)

# drop UNNAMED:0.1
#input_data = input_data.drop('INDEX', axis=1)

df_oct_updated.head()


In [ ]:
col_names = list(df_oct_updated)

# List all column types
#input_data.dtypes
print("Shape of Original data: {}".format(df_oct_updated.shape) + "\n")

input_variables = df_oct_updated.iloc[:,133:221]
print("Variables: " + str(input_variables.columns) + "\n")
print("Shape of variable data: {}".format(input_variables.shape) + "\n")

price = df_oct_updated.iloc[:,-1]

In [ ]:
price

In [ ]:
input_variables.iloc[:,0:-1]

In [ ]:
# Splitting Data into Training and Test
X_train, X_test, y_train, y_test = train_test_split(input_variables.iloc[:,0:-1], price, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

# Applying PCA
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 3)
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)
#explained_variance = pca.explained_variance_ratio_
#print(explained_variance)


# Simple Linear Regression

In [ ]:
# regression libraries
from sklearn.linear_model import LinearRegression

# Fit a linear regression model
lr = LinearRegression().fit(X_train, y_train)
print("lr.coef_{}: ".format(lr.coef_))
print("\n" + "lr_intercept_{}".format(lr.intercept_))

In [ ]:
# Train and Test Performance
print("Training set score: {:2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:2f}".format(lr.score(X_test,y_test)))

# Store Result and Model Name
model_results.append(lr.score(X_test,y_test))
model_name.append("Linear")

# Ridge Regression

In [ ]:
# Coefficients should be close to zero

from sklearn.linear_model import Ridge
ridge = Ridge().fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge.score(X_test, y_test)))

# Store Result and Model Name
model_results.append(ridge.score(X_test,y_test))
model_name.append("Ridge")

In [ ]:
# Try with some handpicked alpha values
# Note: Higher alpha decerease training test performance but increases generalization

ridge_01 = Ridge(alpha = 0.1).fit(X_train, y_train)
print("Training_ridge_0.1 set score: {:.2f}".format(ridge_01.score(X_train, y_train)))
print("Test_ridge_0.1 set score: {:.2f}".format(ridge_01.score(X_test, y_test)))
print("\n")

# Store Result and Model Name
model_results.append(ridge_01.score(X_test,y_test))
model_name.append("Ridge_0.1")

ridge_10 = Ridge(alpha = 10).fit(X_train, y_train)
print("Training_ridge_10 set score: {:.2f}".format(ridge_10.score(X_train, y_train)))
print("Test_ridge_10 set score: {:.2f}".format(ridge_10.score(X_test, y_test)))

# Store Result and Model Name
model_results.append(ridge_10.score(X_test,y_test))
model_name.append("Ridge_10")

# Lasso Regression

In [ ]:
# we have large number of features and only a few of them looked important
from sklearn.linear_model import Lasso
lasso = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(lasso.coef_ != 0)))

# Store Result and Model Name
model_results.append(lasso.score(X_test,y_test))
model_name.append("Lasso")

In [ ]:
# Setting alpha too low will remove the effect of regularization

lasso0001 = Lasso(alpha=0.0001, max_iter=100000).fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso0001.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso0001.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(lasso0001.coef_ != 0)))

# Store Result and Model Name
model_results.append(lasso0001.score(X_test,y_test))
model_name.append("Lasso_0.001")

# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=0.1, l1_ratio=0.7,max_iter = 100000).fit(X_train, y_train)

print("Training set score: {:.2f}".format(enet.score(X_train, y_train)))
print("Test set score: {:.2f}".format(enet.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(enet.coef_ != 0)))

# Store Result and Model Name
model_results.append(enet.score(X_test,y_test))
model_name.append("Elastic Net")

In [ ]:
# # Fitting Multiple Linear Regression to the Training set
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

# # Predicting the Test set results
# y_pred = regressor.predict(X_test)

# from sklearn.metrics import r2_score
# r2_score(y_test,y_pred)
# print("Accuracy: %f"%r2_score(y_test,y_pred))

# # Store Result and Model Name
# model_results.append(r2_score(y_test,y_pred))
# model_name.append("MLR")

# Decision Tree

In [ ]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

# Predicting a new result
y_pred = regressor.predict(X_test)


from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
print("Accuracy of Decision Tree: %f"%r2_score(y_test,y_pred))

# Store Result and Model Name
model_results.append(r2_score(y_test,y_pred))
model_name.append("DT")

# Random Forest

In [ ]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

# Predicting a new result
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
print("Accuracy of Random Forest_10: %f"%r2_score(y_test,y_pred))

# Store Result and Model Name
model_results.append(r2_score(y_test,y_pred))
model_name.append("RF_10")

In [ ]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

# Predicting a new result
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
print("Accuracy of Random Forest_100: %f"%r2_score(y_test,y_pred))

# Store Result and Model Name
model_results.append(r2_score(y_test,y_pred))
model_name.append("RF_100")

In [ ]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)
regressor.fit(X_train, y_train)

# Predicting a new result
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
print("Accuracy of Random Forrest_300: %f"%r2_score(y_test,y_pred))

# Store Result and Model Name
model_results.append(r2_score(y_test,y_pred))
model_name.append("RF_300")

In [ ]:
model_name

# Plot Results of Regression Models

In [ ]:
from matplotlib import cm
import random

# Give each bar separate color
color_code_vector = random.sample(range(1, 100), len(model_name))
reg_color_code_vector = map(lambda x: x/max(color_code_vector), color_code_vector)
reg_color_code_vector = list(reg_color_code_vector)
reg_color_code_vector
colors = cm.hsv(reg_color_code_vector)

# Expressing accuracy in percentage
model_results_percentage = list(map(lambda x: x*100, model_results))

y_pos = np.arange(len(model_name))

plt.rcdefaults() # white background, gets rid of gray
plt.figure(figsize = (30,15))
plt.bar(y_pos, model_results_percentage, align='center', alpha=0.5, width=0.3, color = colors)
plt.xticks(y_pos, model_name, fontsize = 18)
plt.ylabel('Accuracy in perecentage', fontsize=20)
plt.title('Accuracy: Regression Models', fontsize=22, fontweight='bold')
plt.show()